In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.display import clear_output

In [ ]:
# unzip training data
!unzip /kaggle/input/dogs-vs-cats/train.zip
clear_output()

# install imutils
!pip install imutils
clear_output()

# install easy_tfrecord
!pip install easy-tfrecord
clear_output()

In [ ]:
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from imutils import paths
from tfrecord import TFRecord # easy-tfrecord
from sklearn.model_selection import train_test_split

## Get image paths and labels

In [ ]:
# set the directory
data_dir = "./train"

# get all the image paths
image_paths = list(paths.list_images(data_dir))
print(image_paths[:5])

In [ ]:
# extract the labels
labels = [path.split(os.path.sep)[-1].split('.')[0]
          for path in image_paths]
classes = sorted(set(labels))
print("Unique Classes", classes)
print(labels[:5])

## Encode labels

In [ ]:
# encode the labels
label2idx = {label: idx for idx, label in enumerate(classes)}
idx2label = {idx: label for label, idx in label2idx.items()}
labels_enc = [label2idx[label] for label in labels]
print("Labels: ", labels[:5])
print("Labels Encoded: ",labels_enc[:5])

# Split into train, valid & test

In [ ]:
# split the dataset into train, valid, test
X_train, X_valid, y_train, y_valid = train_test_split(image_paths, labels_enc, test_size=0.2,
                                                      random_state=42)

X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.2,
                                                    random_state=42)
len(X_train), len(X_valid), len(X_test)

## Create tf.data Datasets

In [ ]:
# create a function to load the image
def load_images(image_path, label, classes=classes):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.uint8)
    image = image / 255
    
    # onehot encode label
    oh_label = tf.one_hot(label, depth=len(classes))
    return image, oh_label

In [ ]:
# Create tf dataset
train_set = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train))
valid_set = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
test_set = tf.data.Dataset.from_tensor_slices((X_test, y_test))

AUTOTUNE = tf.data.experimental.AUTOTUNE
# apply load_images func
train_set = train_set.map(load_images, num_parallel_calls=AUTOTUNE)
train_set = train_set.batch(32).prefetch(AUTOTUNE)

valid_set = valid_set.map(load_images, num_parallel_calls=AUTOTUNE)
valid_set = valid_set.batch(32).prefetch(AUTOTUNE)

test_set = test_set.map(load_images, num_parallel_calls=AUTOTUNE)
test_set = test_set.batch(32).prefetch(AUTOTUNE)

## Display some images

In [ ]:
fig = plt.figure(figsize=(10, 8))

for batch in train_set.take(1):
    indices = np.random.randint(32, size=8)
    for i,idx in enumerate(indices):
        ax = fig.add_subplot(2, 4, i+1)
        plt.imshow(batch[0][idx])
        plt.axis("off")
        plt.title(idx2label[batch[1][idx].numpy().argmax(axis=-1)])
    plt.tight_layout()
    plt.show()

## Build Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

In [ ]:
class AlexNet:
    @staticmethod
    def build(width, height, depth, classes, reg=0.0002):
        # initialize the model 
        model = Sequential()
        # Block 1
        model.add(Conv2D(96, 11, strides=4, input_shape=(height, width, depth),
                         padding="same", kernel_regularizer=l2(reg), activation="relu"))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        model.add(Dropout(0.25))
        
        # Block 2
        model.add(Conv2D(256, 5, padding="same", kernel_regularizer=l2(reg),
                         activation="relu"))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        model.add(Dropout(0.25))
        
        # Block 3
        model.add(Conv2D(384, 3, padding="same", kernel_regularizer=l2(reg),
                         activation="relu"))
        model.add(BatchNormalization())
        model.add(Conv2D(384, 3, padding="same", kernel_regularizer=l2(reg),
                         activation="relu"))
        model.add(BatchNormalization())
        model.add(Conv2D(256, 3, padding="same", kernel_regularizer=l2(reg),
                         activation="relu"))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        model.add(Dropout(0.25))
        
        # Block 4
        model.add(Flatten())
        model.add(Dense(4096, kernel_regularizer=l2(reg), activation="relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        # Block 5
        model.add(Dense(4096, kernel_regularizer=l2(reg), activation="relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        # Softmax Classifier
        model.add(Dense(classes, kernel_regularizer=l2(reg), activation="softmax"))
        
        return model

## Training with TF Records (easy_tfrecord)
**Writing TFRecords**

In [ ]:
# instantiate object of TFRecord class
tfrec = TFRecord(n_classes=2, image_shape=[224, 224, 3]) # image_shape: all images will be resized to this shape [224, 224]

# compute n_shards for training set (you can directly pass value to n_shards)
train_shards = tfrec.compute_nshards(X_train)
train_tfrecords = tfrec.write_tfrecords(X_train, y_train, save_path="./data/train",
                                        n_shards=train_shards)

# compute n_shards for validation set (optional)
valid_shards = tfrec.compute_nshards(X_valid)
train_tfrecords = tfrec.write_tfrecords(X_valid, y_valid, save_path="./data/valid", 
                                        n_shards=valid_shards)

**Loading from TFRecords**

In [ ]:
# create datasets for training and validation set
train_dataset = tfrec.parse_tfrecord("./data/train")
valid_dataset = tfrec.parse_tfrecord("./data/valid")

# create a function to normalize images: (any kind of preprocessing functions can be added)
def normalize(image, label):
    image = image / 255
    return (image, label)

# normalize images
train_dataset = train_dataset.map(normalize, num_parallel_calls=AUTOTUNE)
valid_dataset = valid_dataset.map(normalize, num_parallel_calls=AUTOTUNE)

**Build and train model**

In [ ]:
# build model
model = AlexNet.build(width=227, height=227, depth=3, classes=2, reg=0.0002)

# define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
# compile model
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# create early stopping callback
early_stop = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# let's track time taken for 5 epochs
start = time.time()
# fit the model
history = model.fit(train_dataset, epochs=5, validation_data=valid_dataset,
                    callbacks=[early_stop])
end = time.time()

In [ ]:
time_taken = round((end - start)/60, 2)
print("Time taken for 5 epochs: {} mins".format(time_taken))

**Uncomment below line if you want to plot losses and accuracies.**

In [ ]:
# plot accuracy and loss
# def plot_metrics(history):
#     fig, ax = plt.subplots(1, 2, figsize=(15, 6))
#     ax[0].plot(history.history["loss"], label="Train Loss")
#     ax[0].plot(history.history["val_loss"], label="Validation Loss")
#     ax[0].set_title("Training and Validation Loss")
#     ax[0].set_xlabel("Epoch")
#     ax[0].set_ylabel("Loss")

#     ax[1].plot(history.history["accuracy"], label="Train Accuracy")
#     ax[1].plot(history.history["val_accuracy"], label="Validation Accuracy")
#     ax[1].set_title("Training and Validation Accuracy")
#     ax[1].set_xlabel("Epoch")
#     ax[1].set_ylabel("Accuracy")
#     plt.suptitle("Trained without TF Records")
#     plt.show()

# plot_metrics(history)

**Evaluate on a test set**

In [ ]:
# evaluate on a test set
loss, acc = model.evaluate(test_set, verbose=0)
print("Loss: {:.4f}\nAccuracy: {:.2f}%".format(loss, acc*100))

**Print classification report**

In [ ]:
from sklearn.metrics import classification_report

predictions = model.predict(test_set)
predictions = predictions.argmax(axis=-1)
print(classification_report(y_test, predictions))

## Without TF Records
Watch time taken per epoch

In [ ]:
# build model
model_1 = AlexNet.build(width=224, height=224, depth=3, classes=2, reg=0.0002)

# define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
# compile model
model_1.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# create early stopping callback
# early_stop = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

start = time.time()
# fit the model
history = model_1.fit(train_set, epochs=5, validation_data=valid_set)
end = time.time()

In [ ]:
time_taken = round((end - start)/60, 2)
print("Time taken for 5 epochs: {} mins".format(time_taken))

In [ ]:
# plot_metrics(history)

In [ ]:
# evaluate on a test set
loss, acc = model_1.evaluate(test_set, verbose=0)
print("Loss: {:.4f}\nAccuracy: {:.2f}%".format(loss, acc*100))

### Time taken to finish 5 epochs:
- **With TFRecords**: _2.63 minutes_
- **Without TFRecords**: _4.05 minutes_